## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [3]:
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
model = ChatGroq(model="llama3-70b-8192", groq_api_key=groq_api_key)

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, my name is Gajalakshmi I am a AI Engineer")])

AIMessage(content="Namaste Gajalakshmi! It's wonderful to meet you! As an AI Engineer, you must be working on some fascinating projects. What kind of AI applications or technologies are you currently focused on? Are you working on natural language processing, computer vision, or something else?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 26, 'total_tokens': 85, 'completion_time': 0.182280852, 'prompt_time': 0.005399386, 'queue_time': 0.715973309, 'total_time': 0.187680238}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'stop', 'logprobs': None}, id='run--8235a2c7-b680-4c03-a84d-57f5b3cd608d-0', usage_metadata={'input_tokens': 26, 'output_tokens': 59, 'total_tokens': 85})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, my name is Gajalakshmi I am a AI Engineer"),
        AIMessage(content="Namaste Gajalakshmi! It's great to meet you, a talented AI Engineer! What an exciting field you're in! Are you working on any specific projects or applications of AI that you'd like to share about?"),
        AIMessage(content="Hey what's my name and what do i do")     
    ]
)

AIMessage(content="?\n\nYou told me earlier that your name is Gajalakshmi and you're an AI Engineer!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 94, 'total_tokens': 117, 'completion_time': 0.068817947, 'prompt_time': 0.011666821, 'queue_time': 0.322399708, 'total_time': 0.080484768}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'stop', 'logprobs': None}, id='run--fe053fff-95f9-4527-94ea-d55f3c0ecff3-0', usage_metadata={'input_tokens': 94, 'output_tokens': 23, 'total_tokens': 117})

Message History we can use a class to wrap our model and make it stateful will keep track the model of input and output

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Gajalakshmi I am a AI Engineer")],
    config=config
)

In [9]:
response.content

"Namaste Gajalakshmi! It's great to meet you! As an AI Engineer, you must be working on some exciting projects that involve building intelligent systems that can think and learn like humans. What specific areas of AI engineering interest you the most? Are you working on natural language processing, computer vision, or perhaps reinforcement learning?"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config
)

AIMessage(content='Your name is Gajalakshmi!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 109, 'total_tokens': 120, 'completion_time': 0.032605133, 'prompt_time': 0.01894414, 'queue_time': 0.249522709, 'total_time': 0.051549273}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'stop', 'logprobs': None}, id='run--75f293b6-c45a-492d-9670-3e4f5e158445-0', usage_metadata={'input_tokens': 109, 'output_tokens': 11, 'total_tokens': 120})

In [11]:
#change the config -->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="what's my name")],
    config=config1
)
response.content

"I'm sorry, but I don't know your name. I'm a large language model, I don't have the ability to know personal information about you, including your name. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to tell me your name, I'd be happy to chat with you!"

Prompt template helps to turn raw user information into a format that the llm can work with In this case the raw user input is just a message which we are passing to the llm let's now make a bit more complicated first let's add in a system message with custom message with custom instructions

In [12]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate(
    [
        ("system","you are an helpful assitant. Answer all the questions to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [13]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Gajalakshmi")]})

AIMessage(content="Namaste Gajalakshmi! It's nice to meet you. That's a beautiful name you have, by the way. Gajalakshmi is a name that symbolizes prosperity and good fortune. It's also associated with the goddess of wealth and good luck in Hindu mythology.\n\nHow can I assist you today, Gajalakshmi? Do you have any questions or topics you'd like to discuss? I'm all ears!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 42, 'total_tokens': 136, 'completion_time': 0.284462791, 'prompt_time': 0.006270074, 'queue_time': 1.899584133, 'total_time': 0.290732865}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'stop', 'logprobs': None}, id='run--6f53bc8f-e4aa-4ef3-bdd4-e0d0178e6ff5-0', usage_metadata={'input_tokens': 42, 'output_tokens': 94, 'total_tokens': 136})

In [14]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [15]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Gajalakshmi")],
    config = config
)
response.content

"Namaste Gajalakshmi! It's a pleasure to meet you. I'm your helpful assistant, here to provide you with assistance and answer any questions you may have. What can I help you with today?"

In [16]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'I remember! Your name is Gajalakshmi.'

In [17]:
#add more complexity
prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are an helpful assitant Answer all the question to the best of ability in {language}.",

        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain= prompt|model

In [18]:
response=chain.invoke({"messages":[HumanMessage(content="My name is Gajalakshmi")],"language":"Hindi"})
response.content

'Namaste Gajalakshmi ji! मैं आपकी मदद के लिए तैयार हूँ। क्या आप कोई प्रश्न पूछना चाहती हैं या कोई विषय पर चर्चा करना चाहती हैं?'

In [19]:
response

AIMessage(content='Namaste Gajalakshmi ji! मैं आपकी मदद के लिए तैयार हूँ। क्या आप कोई प्रश्न पूछना चाहती हैं या कोई विषय पर चर्चा करना चाहती हैं?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 42, 'total_tokens': 104, 'completion_time': 0.192330054, 'prompt_time': 0.005358667, 'queue_time': 0.564630263, 'total_time': 0.197688721}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'stop', 'logprobs': None}, id='run--5ba676a3-3c7f-4a87-9ab3-d805d32356a0-0', usage_metadata={'input_tokens': 42, 'output_tokens': 62, 'total_tokens': 104})

we need to save correct key to use to save the chat history

In [20]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [21]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, I am Gajalakshmi")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते गजलक्ष्मी जी! मैं आपका सहायक हूँ। मैं आपके सभी प्रश्नों का उत्तर देने के लिए यहाँ हूँ। क्या मैं आपकी कोई मदद कर सकता हूँ?'

In [22]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="what my name")],"language":"Hindi"},
    config=config
)

In [23]:
response.content

'आपका नाम गजलक्ष्मी है!'

Managing the conservasation history
one important concept to understand when building the chatbot is how to manage conversational history.if left unmanaged the list of messages the list of message will grow unbounded and potentially overflow the context window of the llm. therefore important step to add a that limits the size of the message you are passing on.

trim message helps to reduce how many messages we are sending to the model. the trimmer allows us to speciy how many tokens we wnat to keep along with parameters like if we want to always keep the system message and whether allow partial messsages

In [24]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\Gajalakshmi G\Documents\Gen-AI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [25]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"I'm not sure! I don't have any information about your personal preferences. But I can ask, what's your favorite ice cream flavor?"

In [26]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2"'

In [27]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [28]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I don't know your name! You didn't tell me earlier, and I don't have any information about you. If you want to share your name, I'd be happy to know!"

In [29]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You didn't ask a math problem yet. This is the beginning of our conversation. If you'd like to ask a math problem, I'm here to help!"